<a href="https://colab.research.google.com/github/kralim/control_c/blob/main/Train_YoloV3_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Connect google drive**

In [1]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Sat Jan 28 15:07:47 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P0    27W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 1				       'My Drive'
'2022-12-30 21-25-26.rar'	        Myy-20220730T145155Z-001.zip
 771627.json			        test.gsheet
 Classroom			       'toq-xyan-ucu - 21 Eyl 2022 (1).gjam'
'Colab Notebooks'		       'toq-xyan-ucu - 21 Eyl 2022.gjam'
'cwa-yzze-zuj - 24 Nis 2022.gjam'       train.gsheet
'evz-rwbb-hmp - 19 Haz 2022 (1).gjam'   Veri-Materyaller
'evz-rwbb-hmp - 19 Haz 2022.gjam'       Win10_21H2_Turkish_x64.zip
 labellenecek			        yolov3
 Mid_dataset_Hafta2.gsheet	       'zoh-wemy-wyf - 29 Oca 2022.gjam'


**1) Clone the Darknet**



In [3]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15502, done.
remote: Total 15502 (delta 0), reused 0 (delta 0), pack-reused 15502
Receiving objects: 100% (15502/15502), 14.17 MiB | 7.39 MiB/s, done.
Resolving deltas: 100% (10403/10403), done.


**2) Compile Darknet using Nvidia GPU**


In [4]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
  946 |                 float rgb[3];
      |                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
 1147 |             if (iteration_

**3) Configure Darknet network for training YOLO V3**

In [5]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [6]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [7]:
# Create folder on google drive so that we can save there the weights
!mkdir "/mydrive/yolov3"

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [8]:
!echo "girilmez" > data/obj.names
!echo -e 'classes= 1\ntrain  = data/train.txt\nvalid  = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data
!mkdir data/ob1j

In [9]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2023-01-28 15:10:31--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  17.9MB/s    in 9.8s    

2023-01-28 15:10:42 (15.8 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [10]:
!unzip /mydrive/yolov3/girilmez.zip -d data/obj

Archive:  /mydrive/yolov3/girilmez.zip
   creating: data/obj/girilmez/
  inflating: data/obj/girilmez/girilmez_0034.png  
  inflating: data/obj/girilmez/girilmez_0042.png  
  inflating: data/obj/girilmez/girilmez_0043.png  
  inflating: data/obj/girilmez/girilmez_0046.png  
  inflating: data/obj/girilmez/girilmez_0047.png  
  inflating: data/obj/girilmez/girilmez_0098.png  
  inflating: data/obj/girilmez/girilmez_0115.png  
  inflating: data/obj/girilmez/girilmez_0124.png  
  inflating: data/obj/girilmez/girilmez_0125.png  
  inflating: data/obj/girilmez/girilmez_0127.png  
  inflating: data/obj/girilmez/girilmez_0135.png  
  inflating: data/obj/girilmez/girilmez_0140.png  
  inflating: data/obj/girilmez/girilmez_0141.png  
  inflating: data/obj/girilmez/girilmez_0156.png  
  inflating: data/obj/girilmez/girilmez_0157.png  
  inflating: data/obj/girilmez/girilmez_0163.png  
  inflating: data/obj/girilmez/girilmez_0164.png  
  inflating: data/obj/girilmez/girilmez_0180.png  
  inflating

In [12]:
# We're going to convert the class index on the .txt files. As we're working with only one class, it's supposed to be class 0.
# If the index is different from 0 then we're going to change it.
import glob
import os
import re

txt_file_paths = glob.glob(r"data/obj/girilmez/*.txt")
for i, file_path in enumerate(txt_file_paths):
    # get image size
    with open(file_path, "r") as f_o:
        lines = f_o.readlines()

        text_converted = []
        for line in lines:
            print(line)
            numbers = re.findall("[0-9.]+", line)
            print(numbers)
            if numbers:

                # Define coordinates
                text = "{} {} {} {} {}".format(0, numbers[1], numbers[2], numbers[3], numbers[4])
                text_converted.append(text)
                print(i, file_path)
                print(text)
        # Write file
        with open(file_path, 'w') as fp:
            for item in text_converted:
                fp.writelines("%s\n" % item)

0 0.764984 0.180535 0.072380 0.097001

['0', '0.764984', '0.180535', '0.072380', '0.097001']
0 data/obj/girilmez/girilmez_0386.txt
0 0.764984 0.180535 0.072380 0.097001
0 0.665837 0.290056 0.041115 0.059392

['0', '0.665837', '0.290056', '0.041115', '0.059392']
1 data/obj/girilmez/girilmez_0402.txt
0 0.665837 0.290056 0.041115 0.059392
0 0.696094 0.292187 0.039062 0.059375

['0', '0.696094', '0.292187', '0.039062', '0.059375']
2 data/obj/girilmez/girilmez_0416.txt
0 0.696094 0.292187 0.039062 0.059375
0 0.526728 0.291992 0.039240 0.058522

['0', '0.526728', '0.291992', '0.039240', '0.058522']
3 data/obj/girilmez/girilmez_0436.txt
0 0.526728 0.291992 0.039240 0.058522
0 0.673689 0.260892 0.057400 0.065406

['0', '0.673689', '0.260892', '0.057400', '0.065406']
4 data/obj/girilmez/girilmez_0317.txt
0 0.673689 0.260892 0.057400 0.065406
0 0.770312 0.204687 0.068750 0.093750

['0', '0.770312', '0.204687', '0.068750', '0.093750']
5 data/obj/girilmez/girilmez_0323.txt
0 0.770312 0.204687 0.06

In [11]:
import glob
images_list = glob.glob("data/obj/girilmez/*.png")
print(images_list)

['data/obj/girilmez/girilmez_0384.png', 'data/obj/girilmez/girilmez_0482.png', 'data/obj/girilmez/girilmez_0438.png', 'data/obj/girilmez/girilmez_0313.png', 'data/obj/girilmez/girilmez_0124.png', 'data/obj/girilmez/girilmez_0450.png', 'data/obj/girilmez/girilmez_0312.png', 'data/obj/girilmez/girilmez_0152.png', 'data/obj/girilmez/girilmez_0410.png', 'data/obj/girilmez/girilmez_0128.png', 'data/obj/girilmez/girilmez_0196.png', 'data/obj/girilmez/girilmez_0506.png', 'data/obj/girilmez/girilmez_0417.png', 'data/obj/girilmez/girilmez_0020.png', 'data/obj/girilmez/girilmez_0341.png', 'data/obj/girilmez/girilmez_0161.png', 'data/obj/girilmez/girilmez_0153.png', 'data/obj/girilmez/girilmez_0322.png', 'data/obj/girilmez/girilmez_0503.png', 'data/obj/girilmez/girilmez_0324.png', 'data/obj/girilmez/girilmez_0398.png', 'data/obj/girilmez/girilmez_0404.png', 'data/obj/girilmez/girilmez_0320.png', 'data/obj/girilmez/girilmez_0382.png', 'data/obj/girilmez/girilmez_0177.png', 'data/obj/girilmez/giril

In [13]:
%pwd

'/content/darknet'

In [14]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

 CUDA-version: 11020 (11060), cuDNN: 8.1.1, GPU count: 1  
 OpenCV version: 4.2.0
yolov3_training
 0 : compute_capability = 750, cudnn_half = 0, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 4, batch = 64, time_steps = 1, train = 1 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   3 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   5 conv    128       3 x 3/ 2    208 x 208 x  64 ->  104 x 104 x 128 1.595 BF
   6 conv     64       1 x 1/ 1    104 x 104 x 128 ->  104 x 104 x  64 0.177 BF
   7 conv    128       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x 128 1.595 BF
   8 Sho